<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/ndl_deeplab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDL 図表自動抽出プログラムの実行例：Google Driveを用いた画像の入力と結果の保存

Google Drive上のフォルダを指定して、当該フォルダに含まれる複数画像に対して物体検出処理を実行し、指定したフォルダに認識結果を出力します。

参考：https://github.com/ndl-lab/tensorflow-deeplab-v3-plus

## 使用方法

- 「1.初期セットアップ」を実行してください。初回のみ実行が必要です。

- 「2. 設定」を変更してください。

- 「3. 実行」を行います。

## 1.初期セットアップ

時間がかかります。初回のみ実行が必要です。

In [ ]:
#@title

%cd /content/

from google.colab import drive
drive.mount('/content/drive/')

!pip install tensorflow==1.15.5

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# ソースコードのダウンロード
!git clone https://github.com/nakamura196/tensorflow-deeplab-v3-plus.git

# 学習済みの重みのダウンロード
%cd tensorflow-deeplab-v3-plus
!wget http://lab.ndl.go.jp/dataset/trainedweights.zip
!unzip -d model50 -o trainedweights.zip

from IPython.display import clear_output 
clear_output()

独自関数のロード

In [ ]:
#@title

import requests
from urllib import request
import json
from tqdm import tqdm
import glob
import bs4

def download_images():
  print("### マニフェストが指定されている場合は、画像のダウンロード ###")
  df = requests.get(manifest).json()
  canvases = df["sequences"][0]["canvases"]
  if process_size > 0:
    canvases = canvases[0:process_size]
  for i in tqdm(range(len(canvases))):
    res = canvases[i]["images"][0]["resource"]
    index = str(i+1).zfill(4)
    path = "{}/img/{}.jpg".format(etc_dir, index)
    os.makedirs(os.path.dirname(path), exist_ok=True)

    if "service" in res:
      info_url = res["service"]["@id"]
      info = requests.get(info_url + "/info.json").json()

      json_path = "{}/json/{}.json".format(etc_dir, index)
      os.makedirs(os.path.dirname(json_path), exist_ok=True)
      json.dump(info, open(json_path, "w"))

      image_width = info["width"]
      image_height = info["height"]

      size = "1600,"

      if image_width < image_height:
        size = ",1600"
      url = "{}/full/{}/0/default.jpg".format(info_url, size)
      
    else:
      url = canvases[i]["images"][0]["resource"]["@id"]
    
    request.urlretrieve(url, path)

def create_manifest():
  print("### マニフェストが指定されている場合は、認識結果をマニフェストファイルに出力 ###")
  files = glob.glob("{}/*.xml".format(output_dir_))

  items = []

  for file in files:
    
    soup = bs4.BeautifulSoup(open(file), 'xml')
    objects = soup.find_all("object")

    # 図表が抽出されなかった場合
    if len(objects) == 0:
      continue

    # 拡張なしのファイル名を取得
    basename = os.path.basename(file)
    file_name = os.path.splitext(basename)[0]

    json_path = "{}/json/{}.json".format(etc_dir, file_name)

    # 2022.04.29時点では、Image Apiに対応している場合のみに実行
    imageApi = True
    if not os.path.exists(json_path):
      continue

    with open(json_path) as f:
      info = json.load(f)

    canvas_id = info["@id"]

    image_width = info["width"]
    image_height = info["height"]

    ratio_width = image_width / 1600
    ratio_height = image_height / 1600

    annos = []

    width, height, index = image_width, image_height, file_name
    
    if imageApi:
      page_img = info["@id"] + "/full/full/0/default.jpg"
      thumb_img = info["@id"] + "/full/200,/0/default.jpg"

      body = {
        "format": "image/jpeg",
        "height": height,
        "type": "Image",
        "width": width,
        "id": info["@id"] + "/full/full/0/default.jpg",
        "service": [{"id": info["@id"], "type": "ImageService2", "profile": "level2"}]
      }

    

    item = {
        "annotations": [
          {
            "id": "{}/annos".format(canvas_id),
            "items": annos,
            "type": "AnnotationPage"
          }
        ],
        "height": height,
        "id": canvas_id,
        "items": [
          {
            "id": "{}/page".format(canvas_id),
            "items": [
              {
                "body": body,
                "id": canvas_id + "/page/imageanno",
                "motivation": "painting",
                "target": canvas_id,
                "type": "Annotation"
              }
            ],
            "type": "AnnotationPage"
          }
        ],
        "label": { "none": ["[{}]".format(index)] },
        "type": "Canvas",
        "width": width,
        "thumbnail": [
          {
            "id": thumb_img,
            "type": "Image",
            "format": "image/jpeg"
          }
        ]
      }
    items.append(item)

    for i in range(len(objects)):
      object_ = objects[i]
      xmin = int(object_.find("xmin").text)
      ymin = int(object_.find("ymin").text)
      xmax = int(object_.find("xmax").text)
      ymax = int(object_.find("ymax").text)

      x = xmin
      y = ymin
      w = xmax - xmin
      h = ymax - ymin

      # 元画像のサイズ
      x = int(x * ratio_width)
      y = int(y * ratio_height)
      w = int(w * ratio_width)
      h = int(h * ratio_height)

      xywh = "{},{},{},{}".format(x, y, w, h)

      anno = {
        "id": canvas_id + "/annos/{}".format(len(annos)),
        "motivation": "commenting",
        "target": canvas_id + "#xywh={}".format(xywh),
        "type": "Annotation",
        "body": { "type": "TextualBody", "value": "graphic" }
      }
      annos.append(anno)

  manifest = {
    "@context": "http://iiif.io/api/presentation/3/context.json",
    "type": "Manifest",
    "label": { "none": ["example"] },
    "items": items
  }

  # マニフェストファイルの出力
  output_path = "{}/manifest.json".format(output_dir_)
  json.dump(manifest, open(output_path, "w"))

  # 以下、確認用にHTMLファイルなどを作成
  json.dump(manifest, open("manifest.json", "w"))

  html = '''
<!DOCTYPE html>
<html>
  <head>
    <meta
      name="viewport"
      content="width=device-width, initial-scale=1, shrink-to-fit=no"
    />
  </head>
  <body>
    <div
      id="mirador"
      style="position: absolute; top: 0; bottom: 0; left: 0; right: 0"
    ></div>
    <script>
      document.write(
        "<script type='text/javascript' src='https://unpkg.com/mirador@latest/dist/mirador.min.js'><\/script>"
      );
    </script>
    <script type="text/javascript">
      var windows = [];

      var manifest = "manifest.json"
      var obj = new Object();
      obj["manifestId"] = manifest;
      obj["thumbnailNavigationPosition"] = "far-right";
      windows.push(obj);

      var windowSettings = {
        highlightAllAnnotations: true,
        sideBarOpen: true,
        defaultSideBarPanel: 'annotations',
        allowClose: false,
        allowMaximize: false,
        allowFullscreen: true,
        hideWindowTitle: false,
      }

      var miradorInstance = Mirador.viewer({
        id: "mirador",
        language : "ja",
        window : windowSettings,
        windows,
      });
    </script>
  </body>
</html>
  '''
  f = open('index.html', 'w')
  f.write(html)
  f.close()

  # サーバの起動
  !nohup python3 -m http.server > /dev/null 2>&1 &
  from google.colab import output

  print("{} にマニフェストファイルを保存しました。".format(output_path))
  print("また以下のリンクから、認識結果を確認できます。")
  output.serve_kernel_port_as_window(8000)

## 2.設定

処理対象のファイルを設定します。

- output_dir: 出力フォルダ

<br/>

入力ファイルの指定方法は2つあります。両方入力された場合、bが優先されます。

<br/>

### a. 画像が格納されたフォルダへのパス

- input_dir: 入力フォルダ

<br/>

### b. IIIFマニフェストURL

- manifest: （任意）IIIFマニフェストファイルのURL。こちらが指定されている場合には、自動的に画像を入力フォルダにダウンロードします。
- process_size: manifestが指定されている場合にのみ有効。処理するcanvas数。-1を指定するとすべてのcanvasを処理対象とする。負荷軽減のため、初期値は5に設定しています。

<br/>

In [ ]:
#@title
output_dir = "/content/drive/MyDrive/ndl_deeplab/output" #@param {type:"string"}
input_dir = "/content/drive/MyDrive/ndl_deeplab/input" #@param {type:"string"}
manifest = "https://www.dl.ndl.go.jp/api/iiif/2558862/manifest.json" #@param {type:"string"}
process_size =  5 #@param {type:"number"}

input_dir = "/content/drive/MyDrive/ndl_deeplab/output/20220429090305/etc/img"
manifest = ""

from pathlib import Path
output_dir = Path(output_dir)
input_dir = Path(input_dir)

## 3.実行

In [ ]:
#@title

import datetime
import pytz

# 時間に基づくIDを生成
run_id = datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y%m%d%H%M%S')
output_dir_ =  "{}/{}".format(output_dir, run_id)

if manifest != "":
  etc_dir = "{}/etc".format(output_dir_)
  input_dir = "{}/img".format(etc_dir)
  download_images()

print("### 推論の実行 ###")
!python picture_extraction.py --input_dir $input_dir --output_dir $output_dir_ --model_dir model50

clear_output()

if manifest != "":
  create_manifest()